In [ ]:
surname = 'Бабижаев'

alp = 'абвгдеёжзийклмнопрстуфхцчшщъыьэюя'
w = [1, 42, 21, 21, 34,  6, 44, 26, 18, 44, 38, 26, 14, 43,  4, 49, 45,
     7, 42, 29,  4,  9, 36, 34, 31, 29,  5, 30,  4, 19, 28, 25, 33]

d = dict(zip(alp, w))
variant = sum([d[el] for el in surname.lower()]) % 40 + 1

print('Задача № 1: ', variant % 3 + 1)
print('Задача № 2: ', variant % 2 + 1)

Задача № 1:  1
Задача № 2:  1


In [33]:
from pyspark.sql import SparkSession
import os
import math
from pyspark.sql.types import DoubleType
from pyspark.sql.functions import expr, mean, stddev, max, explode, split, col, avg

In [3]:
os.environ['JAVA_HOME'] = '/opt/homebrew/opt/openjdk@17/libexec/openjdk.jdk/Contents/Home'
os.environ['PATH'] = f'{os.environ['JAVA_HOME']}/bin:{os.environ['PATH']}'
os.environ['_JAVA_OPTIONS'] = '-Dhadoop.security.authentication=simple -Duser.name=spark'


spark = SparkSession.builder.appName('hw4').getOrCreate()

Picked up _JAVA_OPTIONS: -Dhadoop.security.authentication=simple -Duser.name=spark
Picked up _JAVA_OPTIONS: -Dhadoop.security.authentication=simple -Duser.name=spark
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/11/30 18:18:47 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [6]:
df = spark.read \
    .option('header', 'true') \
    .csv('hw4/movies.csv')

print(df.columns)

['movieId', 'title', 'genres']


In [21]:
SELECTED_GENRES = ['Animation', 'Romance', 'Documentary']

genres_df = df.withColumn('genre', explode(split(col('genres'), "\\|"))) \
    .select('movieId', 'title', 'genre').filter(col('genre').isin(SELECTED_GENRES))

genres_df.show(10)

+-------+--------------------+---------+
|movieId|               title|    genre|
+-------+--------------------+---------+
|      1|    Toy Story (1995)|Animation|
|      3|Grumpier Old Men ...|  Romance|
|      4|Waiting to Exhale...|  Romance|
|      7|      Sabrina (1995)|  Romance|
|     11|American Presiden...|  Romance|
|     13|        Balto (1995)|Animation|
|     15|Cutthroat Island ...|  Romance|
|     17|Sense and Sensibi...|  Romance|
|     25|Leaving Las Vegas...|  Romance|
|     28|   Persuasion (1995)|  Romance|
+-------+--------------------+---------+
only showing top 10 rows



* Задание 1.1

In [22]:
genres_df.groupBy('genre').count().show(10)

+-----------+-----+
|      genre|count|
+-----------+-----+
|    Romance| 1596|
|Documentary|  440|
|  Animation|  611|
+-----------+-----+



* Задание 1.2

In [19]:
ratings_df = spark.read \
    .option('header', 'true') \
    .csv('hw4/ratings.csv')

ratings_df.show(5)

+------+-------+------+---------+
|userId|movieId|rating|timestamp|
+------+-------+------+---------+
|     1|      1|   4.0|964982703|
|     1|      3|   4.0|964981247|
|     1|      6|   4.0|964982224|
|     1|     47|   5.0|964983815|
|     1|     50|   5.0|964982931|
+------+-------+------+---------+
only showing top 5 rows



In [45]:
print(50 * '-')
print('Animation')
ratings_df.join(genres_df, ratings_df.movieId == genres_df.movieId).select('userId',
                                                                           'title', 'genre', col('rating').cast('double')).where(col('genre') == 'Animation').groupBy('title', 'genre').count()\
    .orderBy(col('count').desc())\
    .show(10)

print(50 * '-')
print('Documentary')
ratings_df.join(genres_df, ratings_df.movieId == genres_df.movieId).select('userId',
                                                                           'title', 'genre', col('rating').cast('double')).where(col('genre') == 'Documentary').groupBy('title', 'genre').count()\
    .orderBy(col('count').desc())\
    .show(10)
print(50 * '-')
print('Romance')
ratings_df.join(genres_df, ratings_df.movieId == genres_df.movieId).select('userId',
                                                                           'title', 'genre', col('rating').cast('double')).where(col('genre') == 'Romance').groupBy('title', 'genre').count()\
    .orderBy(col('count').desc())\
    .show(10)

--------------------------------------------------
Animation
+--------------------+---------+-----+
|               title|    genre|count|
+--------------------+---------+-----+
|    Toy Story (1995)|Animation|  215|
|      Aladdin (1992)|Animation|  183|
|Lion King, The (1...|Animation|  172|
|        Shrek (2001)|Animation|  170|
|Beauty and the Be...|Animation|  146|
| Finding Nemo (2003)|Animation|  141|
|Monsters, Inc. (2...|Animation|  132|
|Incredibles, The ...|Animation|  125|
|           Up (2009)|Animation|  105|
|       WALL·E (2008)|Animation|  104|
+--------------------+---------+-----+
only showing top 10 rows

--------------------------------------------------
Documentary
+--------------------+-----------+-----+
|               title|      genre|count|
+--------------------+-----------+-----+
|Bowling for Colum...|Documentary|   58|
|Super Size Me (2004)|Documentary|   50|
|Fahrenheit 9/11 (...|Documentary|   37|
|   Roger & Me (1989)|Documentary|   31|
|  Hoop Dreams (1

* Задание 1.3

In [43]:
print(50 * '-')
print('Animation')
ratings_df.join(genres_df, ratings_df.movieId == genres_df.movieId).select('userId',
                                                                           'title', 'genre', col('rating').cast('double')).where(col('genre') == 'Animation').groupBy('title', 'genre').count().where(col('count') > 10)\
    .orderBy(col('count'))\
    .show(10)

print(50 * '-')
print('Documentary')
ratings_df.join(genres_df, ratings_df.movieId == genres_df.movieId).select('userId',
                                                                           'title', 'genre', col('rating').cast('double')).where(col('genre') == 'Documentary').groupBy('title', 'genre').count().where(col('count') > 10)\
    .orderBy(col('count'))\
    .show(10)
print(50 * '-')
print('Romance')
ratings_df.join(genres_df, ratings_df.movieId == genres_df.movieId).select('userId',
                                                                           'title', 'genre', col('rating').cast('double')).where(col('genre') == 'Romance').groupBy('title', 'genre').count().where(col('count') > 10)\
    .orderBy(col('count'))\
    .show(10)

--------------------------------------------------
Animation
+--------------------+---------+-----+
|               title|    genre|count|
+--------------------+---------+-----+
|Oliver & Company ...|Animation|   11|
|How the Grinch St...|Animation|   11|
|Ponyo (Gake no ue...|Animation|   11|
|All Dogs Go to He...|Animation|   11|
|   Happy Feet (2006)|Animation|   11|
|   Persepolis (2007)|Animation|   11|
|Polar Express, Th...|Animation|   11|
|Meet the Robinson...|Animation|   11|
|Hotel Transylvani...|Animation|   11|
|Rise of the Guard...|Animation|   12|
+--------------------+---------+-----+
only showing top 10 rows

--------------------------------------------------
Documentary
+--------------------+-----------+-----+
|               title|      genre|count|
+--------------------+-----------+-----+
|Thin Blue Line, T...|Documentary|   11|
|Koyaanisqatsi (a....|Documentary|   11|
|   Inside Job (2010)|Documentary|   12|
|King of Kong, The...|Documentary|   12|
|Exit Through the

* Задание 1.4

In [ ]:
print(50 * '-')
print()
ratings_df.join(genres_df, "movieId")\
    .select('title', 'genre', col('rating').cast('double').alias('rating_num'))\
    .where(col('genre') == 'Animation')\
    .groupBy('title', 'genre')\
    .agg(
        sum('rating_num').alias('total_rating'),
        count('*').alias('rating_count')
)\
    .withColumn('manual_avg_rating', col('total_rating') / col('rating_count'))\
    .orderBy(col('manual_avg_rating').desc())\
    .show(10)

--------------------------------------------------



TypeError: unsupported operand type(s) for +: 'int' and 'str'